<h1 style="color: blue; text-align: center;">Multi-task Cascaded Convolutional Networks (MTCNN) for Face Detection and Facial Landmark Alignment Using Parallel</h1>

---

<h2 style="color: gray; text-align: right;">Members of Team:</h2>

| MSSV | Full Name | ID Github |
| ---- | ---- | -------- |
| 1612835 | Bùi Trọng Xuyến | btxuyenhcmus |
| 1612165 | Nguyễn Đào Vinh Hải | NDVHaiHCMUS |
| 1612859 | Nguyễn Đình Hữu | NguyenDinhHuu-HCMUS |


https://github.com/btxuyenHCMUS/MTCNN-GPU.git

---
## Mô tả ứng dụng

**Input:** Một tấm ảnh RGB, Video, Video livestream.

**Output:** Một tấm ảnh RGB hoặc một video live trực tiếp được bounding các khuôn mặt và đánh dấu các vị trí trên khuôn mặt.

### Ý nghĩa thực tế của ứng dụng:
- Bài toán face detection là một bài toán mang lại rất nhiều lợi ích hiện nay: auto focus đối tượng trên máy ảnh, camera giám sát, smart house, tự động phát hiện khuôn mặt để tag trong các mạng xã hội, bước đầu cho bài toán face reccorgnition,...


### Ứng dụng này cần được tăng tốc
- Khi xử lý với video lượng frame đưa xuống quá nhanh, kết quả trả về của mô hình không kịp đáp ứng tạo xử lí nên có độ trễ nhất định.
- Khi chúng ta song song hoá được bài toán này thì sẽ đem lại lợi ích lớn từ việc tiếp kiệm được thời gian.

### Tiềm năng song song hóa
- Song song hóa các phép tích chập trong mô hình (tức là các phép nhân ma trận với nhau).


![index](./images/examples.png)
> Nguồn ảnh: [https://kpzhang93.github.io/MTCNN_face_detection_alignment/](https://kpzhang93.github.io/MTCNN_face_detection_alignment/)

---
## Cài đặt tuần tự

### Ý tưởng

* Các phương pháp xử lý ảnh truyền thống tích chập để rút trích đặt trưng cho việc chọn lọc ra hình ảnh có chứa khuôn mặt. Sử dụng mạng học sâu để huấn luuện các mạng neural cho ra tập các tham số huấn luyện linh động hơn việc đưa ra các tham số cho từng bài toán rút trích đặc trưng cụ thể.
* Thay vì xử lý tất cả các công việc trong cùng một mạng như vgg-16,RCNN,CNN,.. thì trong MTCNN chia ra làm ba giai đoạn thực hiện với những nhiệm vụ khác nhau để cho ra kết quả. Trong đó kết quả của giai đoạn trước sẽ là đầu vào của giai đoạn sau.
* Từ việc xử lí trên từng bức ảnh, nhóm sẽ phát triển lên phát hiện khuôn mặt trên video hoặc video livestream.

**Training data:**
- Nhóm sử dụng network được train sẵn trên bộ dữ liệu WIDER FACE bao gồm 32,203 images và label 393,703 faces.
- Bộ dữ liệu được chia thành các tập training, validation, testing với tỉ lệ 40% / 10% / 50%.

**Đánh giá tính đúng đắn của mô hình**

Để đánh giá tính đúng đắn của mô hình, nhóm của tác giả đã so sánh với các phương pháp khác trong SOTA trên bộ dữ liệu phát hiện khuôn mặt và điểm chuẩn FDDB, WIDER FACE, và dấu mốc trên khuôn mặt được chú thích trong điểm chuẩn AFLW:

- Tập dữ liệu FDDB chứa các chú thích cho 5.171 khuôn mặt trong một tập hợp gồm 2.845 hình ảnh.
- Bộ dữ liệu WIDER FACE bao gồm 393.703 hộp giới hạn khuôn mặt được gắn nhãn trong 32.203 hình ảnh trong đó 50% trong số đó để thử nghiệm (chia thành ba tập con theo độ khó của hình ảnh), 40% dành cho đào tạo và phần còn lại để xác thực.
- AFLW chứa chú thích các mốc khuôn mặt cho 24.386 khuôn mặt và sử dụng cùng một tập hợp con thử nghiệm.


![Evaluation](./images/evaluation.png)

**Evaluation on face alignment**

Nhóm tác giả so sánh performance với các phương pháp RCPR, TSPM, Luxand face SDK, ESR, CDM, SDM, và TCDCN. Sai số trung bình được đo bằng khoảng cách giữa các các bounding box ước tính và ground truth của bounding box, và được chuẩn hóa đối với khoảng cách giữa hai mắt.

![Alignment](./images/alignment.png)

### Các bước thực hiện

#### Sơ đồ thực hiện nhận diện khuôn mặt trong MTCNN

![inputimage](./images/1.png)
> Nguồn ảnh: [https://manutdzou.github.io/2017/01/24/mtcnn.html](https://manutdzou.github.io/2017/01/24/mtcnn.html)


#### Các bước thực hiện

* B1: Cho ảnh đầu vào image pyramid với nhiều kích cỡ khác nhau. (Từ ảnh gốc cho ảnh down scale thành 10 bức ảnh có kích thước nhỏ hơn).

* B2: Input của bước này các bức ảnh đã được xử lí ở pyramid image. Ở trong giai đoạn này P-Net có nhiệm vụ là xác nhận các windows có chứa khuôn mặt nhanh nhưng thiếu chính xác. Sau khi có các bounding box ở P-Net chúng ta sẽ dùng NMS và bounding box regression để giảm số lượng của các bounding box đã được dự đoán ở P-Net. Sau cùng ta có được output là các bounding box và bounding box regression.

![p-net](./images/pnet.png)
> Nguồn ảnh: [https://www.pytorials.com/face-detection-matching-using-facenet/](https://www.pytorials.com/face-detection-matching-using-facenet/)

* B3: Input của bước này là các output bounding box của B2. Ở đây thì R-Net có độ sâu về cấu trúc so với P-Net. Nhiệm vụ chính của R-Net là lọc các bounding box của P-Net + NMS + bounding box regression. Cũng như P-Net sau khi lọc các bounding box ở R-Net thì chúng ta cho qua NMS và bounding box regression để giảm số lượng của các bounding box đã được dự đoán. Ouput ở bước này các bounding box và bounding box regression.

![r-net](./images/rnet.png)
> Nguồn ảnh: [https://www.pytorials.com/face-detection-matching-using-facenet/](https://www.pytorials.com/face-detection-matching-using-facenet/)

* B4: Input ở bước này là các output bounding box của B3. Tương tự như R-Net, nhiệm vụ chính của O-Net là lọc các bounding box. Sau cùng NMS và bounding box regression có nhiệm vụ lọc lại bounding box chính xác và đánh dấu các vị trí trên khuôn mặt được phát hiện. 

![o-net](./images/onet.png)
> Nguồn ảnh: [https://www.pytorials.com/face-detection-matching-using-facenet/](https://www.pytorials.com/face-detection-matching-using-facenet/)


### Đánh giá code tuần tự

#### Đánh giá code tuần tự 

Dưới đây là công thức tính ma trận output của hàm convolution 2d trong pytorch. Nhóm đã dựa vào công thức để viết lại hàm convolution sau đó import vào folder torch. Từ việc, viết hoàn thành hàm convolution, tiếp theo dùng function jit của cuda để chạy chương trình. Dưới đây là kết quả chạy chương trình với hàm convolution 2d với jit function.

![con2d](./images/conv2d.png)

###### Bảng đánh giá code tuần tự


Bảng đánh giá thời gian chạy trên ảnh family.jpeg có kích thước $600x400$

![family](./images/r-jit-family.png)
> Ảnh: family.jpeg

| family.jpeg | P-Net | R-Net | O-Net |
| --- | --- | --- | --- |
| Thời gian chạy | 1460ms | 566ms | 606ms |
| Số lượng bounding box | 192 | 36 | 23 |

Bảng đánh giá thời gian chạy trên ảnh oscar1.jpg có kích thước $4646x1800$

![oscar1](./images/r-jit-oscar.png)
> Ảnh: oscar1.jpg

| oscar1.jpg | P-Net | R-Net | O-Net |
| --- | --- | --- | --- |
| Thời gian chạy | 34200ms | 202000ms | 45500ms |
| Số lượng bounding box | 11081 | 1613 | 967 |

Bảng đánh giá thời gian chạy trên ảnh test.jpg có kích thước $612x408$

![test](./images/r-jit-test.png)
> Ảnh: test.jpg

| test.jpg | P-Net | R-Net | O-Net |
| --- | --- | --- | --- |
| Thời gian chạy | 1510ms | 1580ms | 1450ms |
| Số lượng bounding box | 529 | 80 | 53 |

Bảng đánh giá thời gian chạy trên ảnh test2.jpg có kích thước $768x512$

![test](./images/r-jit-test2.png)
> Ảnh: test2.jpg

| test2.jpg | P-Net | R-Net | O-Net |
| --- | --- | --- | --- |
| Thời gian chạy | 2240ms | 4140ms | 4059ms |
| Số lượng bounding box | 1334 | 258 | 138 |

---
## Cài đặt song song (trên GPU)
### Ý tưởng
**Conv**

- Dense Conv2d,trong mô hình xử dụng nhiều phép tích chập trên một khối họp nhân với nhau.

- Mỗi thread sẽ đảm nhiệm việc tính toán 1 pixel của output.

- Mỗi lớp trong dense sẽ do 1 stream quản lý.

**Prelu**
- Mỗi thread sẽ đảm nhiệm việc bật một pixel dựa vào input và weight.
- Mỗi lớp trong input sẽ do 1 stream quản lý.

**MaxPool**
- Mỗi thread sẽ đảm nhiệm việc tính toán giá trị một pixel
- Mỗi lớp trong dense sẽ do 1 stream quản lý.

### Các bước thực hiện

Dưới đây là một ví dụ minh họa về khối convolution trong MTCNN pytorch

![convolution](./images/conv.png)

Thì theo như hình ở phía trên ta có thể thấy convolution layer gồm n lớp mỗi lớp có kích thước $HxW$. Do đó nhóm sẽ thực hiện chạy một vòng for để duyệt từng lớp convolution trong khối. Trong mỗi lớp convolution sẽ thực hiện song song hóa từng lớp.

Trong từng lớp convolution sẽ thực hiện nhân hai ma trận trên GPU. Mỗi thread trong GPU sẽ xử lí một pixel đầu ra của từng lớp convolution và quá trình đó sẽ thực hiện cho đến khi duyệt hết các lớp convolution trong khối convolution. 

![matrix](./images/matrix.png)

Song khi song song hóa được việc nhân hai ma trận trong từng lớp convolution, nhóm tiếp tục song song hóa được hàm kích hoạt PReLU và hàm maxpool2d theo công thức pytorch 

### Đánh giá code song song

Sau khi thực hiện cuda.jit thì nhóm có một số đánh giá về kết quả như sau:

Kết quả chạy trên ảnh family.jpeg có kích thước 600x400:

> Kết quả ảnh chạy trên cuda.jit:
![cuda](./images/cuda-family.png)
* Thời gian chạy: 1.52 s

Kết quả chạy trên ảnh test.jpg có kích thước 612x408:

![cuda-test](./images/cuda-test.png)
* Thời gian chạy: 1.53 s

Kết quả chạy trên ảnh test.jpg có kích thước 768x512:

![cuda-test2](./images/cuda-test2.png)
* Thời gian chạy: 2.12 s

## Tối ưu hóa song song lần 1

### Ý Tưởng
**Conv**
- Sử dụng shared memory để lưu lại dữ liệu ở input và weight giúp việc tính toán nhanh hơn.

**Prelu**
- Vì việc bật pixel không được gọi lại nhiều lần nên kh cần áp dụng shared memory.

**Maxpool**
- Sử dụng shared memory để lưu lại dữ liệu ở input giúp việc tính toán nhanh hơn.

### Các bước thực hiện

### Đánh giá tối ưu hóa lần 1

## Tối ưu hóa song song lần 2

###  Ý tưởng
- Cả conv, prelu, maxpool đều là dense nên có thể áp dụng streaming để các công việc có thể overlap với nhau

### Các bước thực hiện

### Đánh giá tối ưu hóa lần 2

---
## Nhìn lại quá trình làm đồ án

### Những điều đạt được
- Nâng cao thêm kiến thức về lập trình song song kế thừa từ môn học trước.
- Học được cấu trúc cũng như mô hình thiết kế của một DNN.
- Quản lý công việc, làm việc nhóm và quản lý source code một cách hiệu quả.

### Những điều chưa đạt được
- Chưa thực hiện được tối ưu hóa song song lần 1 và lần 2.
- Quá trình làm việc nhóm đã có trục trặc khi các thành viên không liên kết với nhau.
- Tình hình dịch covid ảnh hưởng đến workflow của nhóm.

---
## Tài liệu tham khảo

- [x] [PDF report](https://arxiv.org/pdf/1604.02878.pdf)
- [x] [Document Guide](https://medium.com/@iselagradilla94/multi-task-cascaded-convolutional-networks-mtcnn-for-face-detection-and-facial-landmark-alignment-7c21e8007923)
- [x] [MTCNN-Pytorch](https://github.com/TropComplique/mtcnn-pytorch)

## Phục lục

**Improvements with previous CNN**

Nhiều CNN đã được thiết kế để nhận diện khuôn mặt. Tuy nhiên, nhóm tác giả nhận thấy rằng hiệu suất của nó có thể bị hạn chế bởi các sự kiện sau:

   - Một số bộ lọc trong các lớp tích chập thiếu tính đa dạng có thể hạn chế khả năng nhận dạng.
   - So với các nhiệm vụ phân loại và phát hiện, phát hiện khuôn mặt là một nhiệm vụ phân loại nhị phân đầy thách thức, do đó, nó có thể cần số lượng bộ lọc trên mỗi lớp ít hơn.

Để làm được điều này, nhóm tác giả giảm số lượng bộ lọc và thay đổi bộ lọc 5 × 5 thành bộ lọc 3 × 3 để giảm tính toán đồng thời tăng độ sâu để có được hiệu suất tốt hơn. Với những cải tiến này, so với kiến ​​trúc trước đó, có thể có được hiệu suất tốt hơn với thời gian chạy ít hơn.

![comparisionspeed](./images/comparisionspeed.png)


#### PL.1. The Three Stages of MTCNN:

**PL.1.1. Stage 1: The Proposal Network (P-Net)**

![P-net](./images/P-net.jpeg)
> P-Net (from MTCNN paper)

* P-net là một Fully convolutional network (FCN). Sự khác biết giữa FCN và CNN đó là FCN không sử dụng dense layer. P-net được sử dụng để lấy các cửa sổ ứng viên và các vector hồi quy bao quanh.

* Bounding box regression là công nghệ phổ biến để dự đoán khu vực của một đối tượng phân lớp, trong trường hợp này là khuôn mặt. Sau khi có tất cả các bounding box thì các bounding box này có thể bị chồng chéo lẫn nhau, cần thông qua một lớp lọc để cho ra khác khu vực không bị trùng lắp.


##### PL.1.2. Stage 2: The Refine Network (R-Net)

![R-net](./images/R-net.jpeg)
> R-Net (from MTCNN paper)

* Tất cả các ứng viên từ P-net sẽ được đưa vào R-net. R-net là một CNN bởi vì có một dense layer ở cuối stage. R-net giảm thêm số lượng ứng viên, thực hiện calibration các bounding box regression và NMS (non-maximum suppression) để hợp nhất các ứng viên trùng lắp.



**PL.1.3. Stage 3: The Output Network (O-Net)**

![O-net](./images/O-net.jpeg)
> R-Net (from MTCNN paper)

* Giai đoạn này tương tự như R-net, nhưng Output của network này mô tả chi tiết hơn về 5 điềm của khuông mặt mắt, mũi, miệng.


#### PL.2. The Three Tasks of MTCNN:

Trong tất cả các stage của Networks đều thực hiện 3 task: face/non-face classification, bounding box regression, facial landmark localization.

**PL.2.1. Face classification:**
* Sử dụng cross-entropy loss vì đây là bài toán phân 2 lớp, với mỗi mẫu $x_i$

$$L_i^{det} = -(y_i^{det}\log(p_i) + (1 - y_i^{det})(1-\log(p_i)))$$

*Trong đó, $p_i$ là xác xuất được tạo ra bớt net chỉ ra một sample là face. $y_i^{det} \in {0, 1}$ là giá trị thật khuôn mặt hoặc không.

**PL.2.2. Bounding box regression:**
Với mỗi cửa sổ ứng viên, chúng ta dự đoán các vị trí tin tưởng tạo nên một bộ offset (left top, height, width). Việc huấn luyện là bài toán regression cho nên sử dụng Euclidean loss cho mỗi sameple $x_i$:

$$L_i^{box} = \parallel\widehat{y}_i^{box} - y_i^{box}\parallel_2^2$$

Trong đó, $\widehat{y}_i^{box}$ là các vị trí dự đoán được của net, $y_i^{box}$ là tọa độ thật. Chúng ta sẽ có bốn góc, bao gồm left top, height, width và $y_i^{box} \in R^4$

**PL.2.3. Facial Landmark localization:**
Tương tự như bài toán regression và chúng ta sẽ đi minimize Euclidean distance:

$$L_i^{landmark} = \parallel\widehat{y}_i^{landmark} - y_i^{landmark}\parallel_2^2$$

## Trả lời issue

### Tại sao phải có 3 stage ?

### Vai trò của R-Net 